## Softnerve - Chapter Spinner

#### Imports

In [51]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter
import gradio as gr
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from openai import OpenAI
import json
from dotenv import load_dotenv
from IPython.display import display, Markdown
import os
import requests
from IPython.display import Image
import gradio as gr
import torch
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq, pipeline
from huggingface_hub import login
import time

#### Initializing Imports

In [52]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")  
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


MODEL = "gemini-2.5-flash"
db_name = "vector_db"

load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")

gemini = OpenAI(
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
    api_key= google_api_key
)

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

hf_token = os.getenv('HF_TOKEN')

#### Setting up the Audio Model

In [53]:
AUDIO_MODEL = "openai/whisper-medium"
processor = AutoProcessor.from_pretrained(AUDIO_MODEL)
speech_model = AutoModelForSpeechSeq2Seq.from_pretrained(
    AUDIO_MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    use_safetensors=True
)

pipe = pipeline(
    "automatic-speech-recognition",
    model=speech_model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch.float16,
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

Device set to use cuda


#### Scrapping the URL

In [54]:
class Website:
    def __init__(self,url):
        self.url = url
        driver.get(url)
        source = driver.page_source
        soup = BeautifulSoup(source, 'html.parser')
        self.title = driver.title if driver.title else "unknown"
        self.image_path = f"{url[-9:]}.png"
        driver.save_screenshot(self.image_path)
        self.screenshot = self.image_path
        for irrelevant in soup.body(['style','script', 'img', 'input']):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n",strip=True)[1100:-500]
        links = [link.get('href') for link in soup.find_all('a',)]
        self.links = [link for link in links if link]
        
    def get_contents(self):
        return f"Title: {self.title}\n\nText: {self.text}"

    def view_screenshot(self):
        if isinstance(self.screenshot, str):
            display(Image(filename=self.screenshot))
        else:
            print("Screenshot path is not valid.")   

#### Style Selection and Chunking

In [55]:
spin_styles = {
    "Simplified": "simplified version",
    "Dramatic": "more dramatic version",
    "Poetic": "poetic and lyrical version",
    "Modern": "modern and casual version",
    "First-Person": "first-person perspective rewrite",
    "Child-Friendly": "version suitable for children aged 10-12",
    "Academic": "formal and academic version",
}

def spin_style_chose(user_choice):
    if user_choice in spin_styles.keys():
        style = spin_styles["Simplified"]
    return f"{user_choice}:{style}"

def chapter_to_chunks(text):
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_text(text)
    return chunks

#### System and User Prompt generation for Writer and Reviewer Agent

In [56]:
writer_system_prompt = """
    You are an AI Writing Agent named 'ChapterSpinner'. 
    Your job is to rewrite book chapters in different styles while preserving the original meaning, character emotions, and storyline.
    Be creative but do not add or remove plot points.
"""

def writer_user_prompt(user_choice, chapter_content):
    writer_user_prompt_template = f"""
        Rewrite the following chapter into a {spin_style_chose(user_choice)}. 
        Keep the core meaning, storyline, and details intact, but change the tone, sentence structure, and language to match the target style. 
        Do not invent events or characters.
        === ORIGINAL CHAPTER ==={chapter_content}
    """
    return writer_user_prompt_template

reviewer_system_prompt = """
    You are an AI Reviewer Agent named 'ChapterPolisher'.
    Your role is to refine AI-generated rewritten chapters for grammar, coherence, and tone consistency without altering the intended style or meaning.
"""

def reviewer_user_prompt(user_choice, rewritten_chapter):
    reviewer_user_prompt_template = f"""
    Refine the following rewritten chapter to improve grammar, sentence flow, and tone consistency while maintaining its {spin_style_chose(user_choice)}.
    Make the transitions smoother and ensure clarity and readability. Do not change the storyline or remove meaningful content.
    === REWRITTEN CHAPTER ===\n{rewritten_chapter}
    """
    return reviewer_user_prompt_template

#### Creating a versions.json file to store Versions and Rewards

In [57]:
def versioning(stars, reviewed_chapter, updated_chapter, chapter_no):
    import json, os
    version_name = f"version_{int(time.time())}"  # use timestamp to ensure uniqueness
    reward = -1 if int(stars) <= 2 else (0 if int(stars) == 3 else 1)

    if updated_chapter:
        version_entry = {
        "Version": version_name,
        "Content": updated_chapter,
        "Reward": reward
    }
    else:
    
        version_entry = {
            "Version": version_name,
            "Content": reviewed_chapter,
            "Reward": reward
        }

    filename = "versions.json"

    if os.path.exists(filename):
        with open(filename, 'r') as f:
            all_versions = json.load(f)
    else:
        all_versions = {}

    chapter_key = f"Chapter{chapter_no}"

    if chapter_key not in all_versions:
        all_versions[chapter_key] = []

    all_versions[chapter_key].append(version_entry)

    with open(filename, 'w') as f:
        json.dump(all_versions, f, indent=4)

    print(f"Version for {chapter_key} saved successfully.")
    return f"Saved {version_name} for {chapter_key} with reward: {reward}"

    

#### Setting up the connection between Both the Agents

In [58]:
chapter = None  
conversation_chain = None  
reviewed_chapter = ""
final_chapter = ""


def handle_initial_inputs(book_no, chapter_no, style_choice):
    global chapter, reviewed_chapter
    if int(book_no) < 4:
        if int(book_no) ==1:
            if int(chapter_no) >= 14:
                return "Invalid chapter number. Must be less than 14.", "", ""
        elif int(book_no) ==2:
            if int(chapter_no) >= 10:
                return "Invalid chapter number. Must be less than 10.", "", ""
        elif int(book_no) ==3:
            if int(chapter_no) >= 9:
                return "Invalid chapter number. Must be less than 9.", "", ""
        elif int(book_no) ==4:
            if int(chapter_no) >= 13:
                return "Invalid chapter number. Must be less than 13.", "", ""
        else:
            return "Invalid book number. Must be less than 5.", "", ""
            
    url = f"https://en.wikisource.org/wiki/The_Gates_of_Morning/Book_{book_no}/Chapter_{chapter_no}"
    chapter = Website(url)

    content = chapter.get_contents()

    # Writer + Reviewer flow
    writer_message = [
        {"role": "system", "content": writer_system_prompt},
        {"role": "user", "content": writer_user_prompt(style_choice, content)}
    ]
    writer_response = gemini.chat.completions.create(model=MODEL, messages=writer_message)
    rewritten_chapter = writer_response.choices[0].message.content

    reviewer_message = [
        {"role": "system", "content": reviewer_system_prompt},
        {"role": "user", "content": reviewer_user_prompt(style_choice, rewritten_chapter)}
    ]
    reviewer_response = gemini.chat.completions.create(model=MODEL, messages=reviewer_message)
    reviewed_chapter = reviewer_response.choices[0].message.content

    update_vectorstore(reviewed_chapter)
    return "Chapter Reviewed", reviewed_chapter, reviewed_chapter 

#### Applying Human in the Loop

In [59]:
def apply_instruction(instruction_text, current_text):
    prompt = f"Apply the following instruction to the reviewed chapter: '{instruction_text}'\n\nChapter:\n{current_text}"
    response = gemini.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}]
    )
    new_text = response.choices[0].message.content
    update_vectorstore(new_text)
    return new_text

#### Generating Embeddings and Defining vectordatabase for each Chapter generation

In [60]:
def update_vectorstore(final_text):
    global conversation_chain
    chunks = chapter_to_chunks(final_text)

    if os.path.exists(db_name):
        Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

    vectorstore = Chroma.from_texts(chunks, embedding=embeddings)
    llm = ChatGoogleGenerativeAI(temperature=0.7, model=MODEL)
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 25})
    conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)
    print("RAG pipeline initialized successfully.")

#### Setting up Screenshot

In [61]:
def get_image(book_no, chapter_no):
    try:
        chapter_no = int(chapter_no)
        if int(book_no) < 4:
            if int(book_no) ==1:
                if int(chapter_no) >= 14:
                    return "Invalid chapter number. Must be less than 14.", "", ""
            elif int(book_no) ==2:
                if int(chapter_no) >= 10:
                    return "Invalid chapter number. Must be less than 10.", "", ""
            elif int(book_no) ==3:
                if int(chapter_no) >= 9:
                    return "Invalid chapter number. Must be less than 9.", "", ""
            elif int(book_no) ==4:
                if int(chapter_no) >= 13:
                    return "Invalid chapter number. Must be less than 13.", "", ""
            else:
                return None
    except:
        return None

    url = f"https://en.wikisource.org/wiki/The_Gates_of_Morning/Book_{book_no}/Chapter_{chapter_no}"
    chapter = Website(url)

    return chapter.screenshot

#### Setting up a Pipeline to handle chat

In [62]:
def handle_chat(message, history):
    if history is None:
        history = []
        
    if conversation_chain is None:
        return history + [[message, "Please enter chapter number first."]]
        
    print("conversation_chain exists:", conversation_chain is not None)

    try:
        result = conversation_chain.invoke({"question": message})
        bot_response = result["answer"]
        history.append([message, bot_response]) 
    except Exception as e:
        history.append([message, f"Error: {str(e)}"])

    return history

#### Function to transcribe Audio to Text

In [63]:
def process_audio(audiofile):
    try:
        if audiofile is None:
            return "No audio file received."

        print("[INFO] Transcribing:", audiofile)
        result = pipe(audiofile, return_timestamps=True)
        transcription = result["text"]
        print("[INFO] Transcription complete:", transcription[:100])
        return transcription
    
    except Exception as e:
        print("[ERROR] Failed to transcribe audio:", str(e))
        return f"Error: {str(e)}"

#### Setting up a Pipeline to handle voice based chat

In [64]:
def voice_to_chat(audio_path, chat_history=None):
    if chat_history is None:
        chat_history = []

    transcription = process_audio(audio_path)

    if conversation_chain is None:
        chat_history.append([transcription, "Please enter chapter number first."])
    else:
        try:
            result = conversation_chain.invoke({"question": transcription})
            bot_response = result["answer"]
            chat_history.append([transcription, bot_response])
        except Exception as e:
            chat_history.append([transcription, f"Error: {str(e)}"])

    return chat_history

#### Gradio Interface for better interaction

In [65]:
with gr.Blocks() as demo:
    gr.Markdown("## AI-Powered Chapter Spinner")

    with gr.Row():
        book_no = gr.Textbox(label="Enter Book Number (1-4)", value=None)
        chapter_no = gr.Textbox(label="Enter Chapter Number", value=None)
        style_choice = gr.Dropdown(choices=list(spin_styles.keys()), label="Choose Rewrite Style", value="Simplified")
        submit_button = gr.Button("Generate Chapter")

    with gr.Row():
        
        image_output = gr.Image(label="Scrapped Image")

    
    status = gr.Textbox(label="Status Message", interactive=False)
    reviewed_output = gr.Textbox(label="Reviewed Chapter Output", lines=20)

    with gr.Row():
        manual_edit_box = gr.Textbox(label="Edit Chapter Manually", lines=15)
        instruction_box = gr.Textbox(label="Or Give Instructions to Improve")

    apply_instruction_btn = gr.Button("Apply Instruction")

    updated_output = gr.Textbox(label="Final Chapter After Instruction or Manual Edits", lines=20)

    chatbot = gr.Chatbot()
    user_query = gr.Textbox(label="Ask Questions About the Chapter")
    with gr.Row():
        audio_input = gr.Audio(type="filepath", label="Upload or Record Audio")
        
        
        transcribe_button = gr.Button("Transcribe")
        ask_btn = gr.Button("Ask")
        
    rating = gr.Radio(choices=["1", "2", "3", "4", "5"], label="Rate the AI's Output", interactive=True)
    rate_button = gr.Button("Submit Rating")
    status = gr.Textbox(label="Status Message", interactive=False)

    submit_button.click(handle_initial_inputs, inputs=[book_no, chapter_no, style_choice], outputs=[status, reviewed_output, manual_edit_box])
    chapter_no.change(fn=get_image, inputs=[book_no, chapter_no], outputs=image_output)
    apply_instruction_btn.click(apply_instruction, inputs=[instruction_box, manual_edit_box], outputs=updated_output)
    transcribe_button.click(fn= voice_to_chat, inputs=audio_input, outputs=chatbot)
    ask_btn.click(handle_chat, inputs=user_query, outputs=chatbot)
    rate_button.click(
    fn=versioning,
    inputs=[rating, reviewed_output, updated_output, chapter_no],
    outputs=status
    )



demo.launch(inbrowser=True)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_36764\860229548.py:26: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()
D:\anaconda\envs\gpu_env\lib\site-packages\gradio\utils.py:1074: UserWarning: Expected 2 arguments for function <function handle_chat at 0x0000020EAF6CBEB0>, received 1.
  warnings.warn(
D:\anaconda\envs\gpu_env\lib\site-packages\gradio\utils.py:1078: UserWarning: Expected at least 2 arguments for function <function handle_chat at 0x0000020EAF6CBEB0>, received 1.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.


RAG pipeline initialized successfully.
RAG pipeline initialized successfully.


D:\anaconda\envs\gpu_env\lib\site-packages\gradio\helpers.py:1031: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")


conversation_chain exists: True
